Parse Text Data, make Train Split
---

In [113]:
import glob
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

def parse_lyrics():
    unique_line_ratios = []
    artists = []
    titles = []
    genres = []
    lyric_strings = []
    
    #iterate through files
    for filename in glob.iglob('data/*.txt'):
        genre = filename[5:len(filename)-4]
        numberOfSongs = 0
        file = open(filename)
        lines = file.readlines()
        i = 0
        while i < len(lines) and not lines[i].startswith("###"):
            i = i + 1
        while i < len(lines):
            numberOfSongs = numberOfSongs + 1
            artists.append(lines[i][3:].strip())
            titles.append(lines[i+1][3:].strip())
            i = i + 2
            #extract features from song
            
            lyrics = []
            while i < len(lines) and not lines[i].startswith("###"):
                line = lines[i].strip()
                if line != "":
                    lyrics.append(line)
                i = i + 1
                
            lyric_strings.append(make_lyric_string(lyrics))   
            unique_line_ratios.append(calc_unique_line_ratio(lyrics))
            genres.append(genre)    
    
    return unique_line_ratios, artists, titles, genres, lyric_strings;

def calc_unique_line_ratio(lyrics):
    return len(set(lyrics)) / len(lyrics)

def make_lyric_string(lyrics):
    lyric_string = ''
    for l in lyrics:
        lyric_string += l+' '
    return lyric_string 

unique_line_ratios, artists, titles, genres, raw_lyrics = parse_lyrics()
vectorizer = CountVectorizer()
bag_of_words_lyrics = vectorizer.fit_transform(raw_lyrics)

X_train, X_test, y_train, y_test = train_test_split(bag_of_words_lyrics, genres, test_size=0.2, random_state=42)

Classify & Evaluate
---

In [121]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

bow_cls = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
bow_pred = bow_cls.predict(X_test)

print ("BOW Classifier, F1 micro: " + str(f1_score(pred, y_test, average='micro')))
print("BOW Classifier, F1 macro: " + str(f1_score(pred, y_test, average='macro')))

BOW Classifier, F1 micro: 0.738636363636
BOW Classifier, F1 macro: 0.741586583351
